# CSS 201 / 202 - CSS Bootcamp

## Week 06 - Lecture 04

### Umberto Mignozzetti

# Machine Learning

In [ ]:
## Loading the relevant packages
import pandas as pd
import numpy as np

# Plotting things:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Look at our friend here to help with GAM
import statsmodels.api as sm
from statsmodels.gam.api import GLMGam, BSplines

# Loading scikit learn relevant packages (note our new friends!)
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score, get_scorer_names, mean_squared_error, r2_score, mean_squared_error, roc_auc_score, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score, KFold, GridSearchCV
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree, export_text
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, SplineTransformer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import VotingClassifier, BaggingRegressor, BaggingClassifier, RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor, GradientBoostingClassifier

## Chile Dataset

In [ ]:
## Loading Chile data
chile = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/chilesurvey.csv')
chile_clean = chile.dropna()
chile_clean = chile_clean[chile_clean['vote'].isin(['Y', 'N'])]
chile_clean['vote'] = np.where(chile_clean['vote'] == 'Y', 1, 0)
chile_clean['logincome'] = np.log(chile_clean['income'])
chile_clean['logpop'] = np.log(chile_clean['population'])
dummies = pd.get_dummies(chile_clean['sex'], prefix = 'sex', drop_first = True)
chile_clean = pd.concat([chile_clean, dummies], axis=1)
dummies = pd.get_dummies(chile_clean['region'], prefix = 'region', drop_first = True)
chile_clean = pd.concat([chile_clean, dummies], axis=1)
dummies = pd.get_dummies(chile_clean['education'], prefix = 'education', drop_first = True)
chile_clean = pd.concat([chile_clean, dummies], axis=1)
chile_clean.head()

## Education Dataset

In [ ]:
## Education Expenditure Dataset
educ = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/educexp.csv')
y = educ['education']
X = educ[['income', 'young', 'urban']]

## Polynomials
for power in range(2, 4):
    for var in ['income', 'young', 'urban']:
        X[var + '_' + str(power)] = X[var] ** power
        
## Standardizing the X variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Lasso Regression

## Lasso Regression

- Ridge regression has one disadvantage: it most of the time includes $p$ predictors.

- The shrinkage never sets coefficients to be exactly zero (that is, be removed from the prediction).

- This could potentially make subset selection better than ridge.

## Lasso Regression

- But one alternative, using the same principles as the ridge regression is the **lasso** regression.

- In the lasso regression, the objective function becomes:

$$ RSS + \alpha \sum_{j=1}^p|\beta_j| $$

- Does the same as ridge: the larger the $\alpha$, the more the *shrinkage*.

## Lasso Regression

- Unlike ridge, for some values of $\alpha$, **lasso** actually force coefficients to be exactly equal to zero.

- Thus, **lasso** performs variable selection, much like the subset selection models we have seen.

- **Side-effect**: Makes models easier to interpret!
    + Yields *sparse* models: models that only involve a subset of the variables.
    
- Like ridge, selecting a good $\alpha$ is critical.

## Lasso Regression

In [ ]:
## Lasso Regression (badly done)
lasso = Lasso(alpha = 1).fit(X, y)
print('The R-squared for this regression is: ' + str(lasso.score(X, y)))
plt.bar(X.columns, lasso.coef_)
plt.xticks(rotation=45)
plt.show()

## Lasso Regression

In [ ]:
## Lasso Regression (greatly done)
lasso = Lasso(alpha = 1).fit(X_scaled, y)
print('The R-squared for this regression is: ' + str(lasso.score(X_scaled, y)))
plt.bar(X.columns, lasso.coef_)
plt.xticks(rotation=45)
plt.show()

## Lasso (the book calls the regularization parameter $\lambda$) 

![img](https://github.com/umbertomig/POLI175public/blob/main/img/lasso1.png?raw=true)

## Lasso Regression

In [ ]:
## Example by Kornel Kielczewski in the sklearn Ridge documentation, adapted by me.
lasso = Lasso(max_iter = 20000000)
coefs = list()
errors = list()
alphas = np.logspace(-6, 6, 50)

for a in alphas:
    lasso.set_params(alpha = a).fit(X_scaled, y)
    coefs.append(lasso.coef_)
    errors.append(np.mean((lasso.predict(X_scaled) - y) ** 2))

coefs = pd.DataFrame(coefs, columns = X.columns, index = alphas)
print(errors[0:5])
coefs.head()

## Lasso Regression

In [ ]:
# Lasso coefficients as a function of the regularization paramenter alpha
g = sns.lineplot(data = coefs)
g.set(xscale='log')
plt.show()

## Lasso Regression

In [ ]:
# Lasso Mean Squared Error as a function of the regularization paramenter alpha
g = sns.lineplot(x = alphas, y = errors)
g.set(xscale='log')
plt.show()

## Lasso x Ridge Regression (the book calls the regularization parameter $\lambda$) 

- Selection property of lasso: 
    + Lasso and ridge are equivalent to a constraint on the shape of the acceptable parameter space.
    + But the "diamond shape" of lasso makes it shrinks some coefficients towards zero.

![img](https://github.com/umbertomig/POLI175public/blob/main/img/lassovsridge3.png?raw=true)

## Lasso x Ridge Regression (the book calls the regularization parameter $\lambda$) 

- Lasso performs similarly to ridge in most cases. In these cases, I'd say that lasso is better:
    + Reduces the complexity in the model.

![img](https://github.com/umbertomig/POLI175public/blob/main/img/lassovsridge1.png?raw=true)

## Lasso x Ridge Regression (the book calls the regularization parameter $\lambda$) 

- But when all coefficients are different from zero, then ridge is better.

![img](https://github.com/umbertomig/POLI175public/blob/main/img/lassovsridge2.png?raw=true)

## Lasso x Ridge Regression

- To summarize, none is better in all situations.

- You may need to search which model is better.

- Moreover, finding $\alpha$ is also a big deal. Cross-validation can help us with that!

## Cross-Validation

- To select the tuning parameters you can use cross-validation.

- The idea is to search through a grid of tuning parameter candidates, selecting the one that does best in the cross-validation.

- It is indeed a very straight-forward idea, if you think about it.

## Cross-Validation

In [ ]:
## Lasso with Cross-Validation
lasso = Lasso(max_iter = 20000000)
coefs = list()
errors = list()
CVerrors = list()
alphas = np.logspace(-6, 6, 50)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 12345)

for a in alphas:
    lasso.set_params(alpha = a).fit(X_train, y_train)
    CVerrors.append(np.mean((lasso.predict(X_test) - y_test) ** 2))
    errors.append(np.mean((lasso.predict(X_train) - y_train) ** 2))

print('The alpha that minimizes the Lasso Cross-Validation MSE is: ' + str(alphas[CVerrors.index(min(CVerrors))]))
    
mses = pd.DataFrame({
    'trainMSE': errors,
    'testMSE': CVerrors}, index = alphas)

## Cross-Validation

In [ ]:
# Cross Validation Lasso MSE as a function of the regularization parameter alpha
g = sns.lineplot(data = mses)
g.set(xscale='log')
plt.show()

# Beyond Linearity

## Beyond Linearity

- So far, we have focused on linear models.

- Most of the time, linear approximations are excellent:
    + Easy to interpret
    + Easy to run

- And all the GLM flavors afford lots of flexibility regarding how we deal with data.

- We will now relax the linearity assumption but keep it as simple as possible.

## Polynomial Regression

- Expands the default model ($y_i = \beta_0 + \beta_1x_i + \varepsilon_i$) to consider a polynomial $d$:

$$ y_i = \beta_0 + \beta_1x_i + \beta_2x_i^2 +\cdots + \beta_dx_i^d + \varepsilon_i $$
                     
- Lots of flexibility here, but we seldom use $d>4$ because then it gets *excessively* flexible.

- Prediction very straightforward:

$$ \hat{f}(x_0) = \hat{\beta}_0 + \hat{\beta}_1x_0 + \hat{\beta}_2x_0^2 +\cdots + \hat{\beta}_dx_0^d $$

## Polynomial Regression

![img](https://github.com/umbertomig/POLI175public/blob/main/img/polyreg1.png?raw=true)

## Polynomial Regression

In [ ]:
## Create the polynomials (with interaction terms!)
poly = PolynomialFeatures(degree = 4)
X = educ.reset_index()['income'].to_frame()
X = poly.fit_transform(X)
print(list(poly.get_feature_names_out(['income'])))

## Polynomial Regression

In [ ]:
## Fit a linear regression
reg = LinearRegression().fit(X, y)
print('The MSE for a quartic polynomial is: ' + str(np.mean((reg.predict(X) - y) ** 2)))

## Polynomial Regression

In [ ]:
# For the fully-specified model (lots of interactions...)
poly = PolynomialFeatures(degree = 4)
X = educ.reset_index()[['income', 'urban', 'young']]
X = poly.fit_transform(X)
print(list(poly.get_feature_names_out(['income', 'urban', 'young'])))
## Fit a linear regression
reg = LinearRegression().fit(X, y)
print('\n\nThe MSE for a quartic polynomial on income, urban, and \nyoung + interactions, is: ' + str(np.mean((reg.predict(X) - y) ** 2)))

## Polynomial Regression

In [ ]:
## With cross-validation now

# Only income
X = educ.reset_index()['income'].to_frame()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 12345)
X_train = poly.fit_transform(X_train)
X_test = poly.fit_transform(X_test)
reg = LinearRegression().fit(X_train, y_train)
print('The CV-MSE for a quartic polynomial on income is: ' + str(np.mean((reg.predict(X_test) - y_test) ** 2)))

## Polynomial Regression

In [ ]:
## With cross-validation now

# For the fully-specified model (lots of interactions...)
X = educ.reset_index()[['income', 'urban', 'young']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 12345)
X_train = poly.fit_transform(X_train)
X_test = poly.fit_transform(X_test)
reg = LinearRegression().fit(X_train, y_train)
print('The CV-MSE for a quartic polynomial on income, urban, and \nyoung + interactions, is: ' + str(np.mean((reg.predict(X_test) - y_test) ** 2)))

## Piecewise-constant Regression

- Here, we would be breaks in the predictor, making it an ordered categorical variable instead of a continuous variable.

- Let a variable $X$, the indicator function $I(.)$, and a set of cutpoints $\{c_1, c_2, \cdots, c_k\}$. The stepped variable $X$, $C_j(X)$, becomes:

$$
\begin{align}
C_{0}(X) &= I(X < c_1) \\
C_{1}(X) &= I(c_1 \leq X < c_2) \\
C_{2}(X) &= I(c_2 \leq X < c_3) \\
&\vdots\\
C_{K-1}(X) &= I(c_{K-1} \leq X < c_K) \\
C_{K}(X) &= I(X \geq c_K)
\end{align}
$$

## Piecewise-constant Regression

- Example use: when age is defined in 5-years bins.

- Detail: $C_0(x_i) + C_1(x_i) + \cdots + C_K(x_i) = 1$. This would make adding all pieces impossible:
    + Perfect Collinearity.

- The regression model becomes:

$$ y_i = \beta_0 + \beta_1C_1(x_i) + \beta_2C_2(x_i) +\cdots + \beta_KC_K(x_i) + \varepsilon_i $$

## Piecewise-constant Regression

![img](https://github.com/umbertomig/POLI175public/blob/main/img/pcreg1.png?raw=true)

## Piecewise-constant Regression

In [ ]:
## Piecewise-constant Regression
X = educ.reset_index()['income'].to_frame()
X = pd.cut(X.income, bins = [0, 2500, 3000, 3200, 3500, 4000, 5000])
X = pd.get_dummies(X, drop_first = True)
reg = LinearRegression().fit(X, y)
print('The MSE for this piecewise constant regression is: ' + str(np.mean((reg.predict(X) - y) ** 2)))

## Polynomial Regression

In [ ]:
## With cross-validation now
X = educ.reset_index()['income'].to_frame()
X = pd.cut(X.income, bins = [0, 2500, 3000, 3200, 3500, 4000, 5000])
X = pd.get_dummies(X, drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 12345)
X_train = poly.fit_transform(X_train)
X_test = poly.fit_transform(X_test)
reg = LinearRegression().fit(X_train, y_train)
print('The CV-MSE for this piecewise constant regression is: ' + str(np.mean((reg.predict(X_test) - y_test) ** 2)))

# Tree-based methods

## Tree-based methods

- Tree-based methods consist of segmenting the predictors' space into many regions

- Then, use these regions to predict the target variable.
    + We use heuristics here, such as the variable's mean in the region.
    
- This approach is called the `decision tree method`

- It, by itself, is terrible. But then many methods improve the efficiency considerably
    + At the cost of interpretability

## Decision Trees

- Can both be applied to regression and classification.

- They look like this:

![img](https://github.com/umbertomig/POLI175public/blob/main/img/tree1.png?raw=true)

## Decision Trees

- This is how they segment the space:

![tree2](https://github.com/umbertomig/POLI175public/blob/main/img/tree2.png?raw=true)

## Decision Trees

- Formally:
    1. We divide the predictors' space into distinct and non-overlapping regions $R_1$, $R_2$, $\cdots$, $R_J$.
    2. For all observations within $R_j$, we make the same prediction.
    
- How to construct the $R_1, R_2, \cdots, R_J$ space?

## Decision Trees

- We minimize the RSS of the following equation:

$$ \sum_{j=1}^J\sum_{i \in R_j} (y_i - \hat{y}_{R_j})^2 $$

- But it is easy to see that solving this is computationally infeasible.
    + Multiple variables would amount to multiple regions.
    + Many combinations and regions would be possible.

## Decision Trees

- The way we do this is called the `greedy` approach:

1. We consider one variable at a time.

2. We split it recursively.

3. We consider the regions that divide the space into two half-spaces:

$$ R_1(j, s) = \{X | X_j < s\} \quad and \quad R_2(j, s) = \{X | X_j \geq s\} $$

## Decision Trees

- Minimizing:

$$ \sum_{i: \ x_i \in R_1(j,s)} (y_i - \hat{y}_{R_1})^2 + \sum_{i: \ x_i \in R_2(j,s)} (y_i - \hat{y}_{R_2})^2$$

- With $\hat{y}_{R_j}$ the mean of $y$ in the $j$-th half-space.

## Decision Trees

- This could be the optimal, but never the greedy approach:

![tree3](https://github.com/umbertomig/POLI175public/blob/main/img/tree3.png?raw=true)

## Decision Trees

- This represents the greedy approach:

![tree3](https://github.com/umbertomig/POLI175public/blob/main/img/tree4.png?raw=true)

## Decision Trees

- And this is the levels / the regression tree:

![tree5](https://github.com/umbertomig/POLI175public/blob/main/img/tree5.png?raw=true)

## Decision Trees

- This approach will likely overfit the data. We need them to `prune` our tree a bit.

- This involves removing parts of our tree that are not being helpful in the testing set predictions.

- One heuristic is to use the size of the tree to penalize it. This is called `cost complexity pruning`

For all $T \subset T_0$:

$$ \sum_{m=1}^{|T|} \sum_{i: \ x_i \in R_m} (y_i - \hat{y}_{R_m})^2 + \alpha |T| $$

## Decision Trees

**Regression Tree Algorithm:**

1. Use recursive binary split (greedy approach) to grow a large tree on the training data. Stop when the terminal node reaches fewer than some threshold minimum number of observations.

2. Apply the cost-complexity pruning to the large tree to obtain a sequence of trees $T_1 \subset T_2 \subset \cdots \subset T_0$, as a function of $\alpha$.

## Decision Trees

**Regression Tree Algorithm:**

3. Use K-fold cross-validation to choose $\alpha$, i.e., for each $k \in \{1, 2, \cdots, K\}$:
    - Repeat steps 1 and 2 on all but $k$th fold of training data
    - Compute the MSE in the left-out fold
    - Pick $\alpha$ that minimizes the MSE.

4. Fit the corresponding tree for the optimal $\alpha$.

## Decision Trees

![tree6](https://github.com/umbertomig/POLI175public/blob/main/img/tree6.png?raw=true)

## Classification Trees

- Very similar, but you change the prediction and the statistic we look at.

1. We predict that all classes belong to the `most commonly occurring class` in the data.

2. We look at the `classification error rates` to grow our trees.

## Classification Trees

- Let $\hat{p}_{mk}$ the proportion of cases in the $m$-th region that belong to the $k$-th class.

- The error rate is:

$$ E = 1 - \max_k (\hat{p}_{mk}) $$

## Classification Trees

- But, the classification error is sensitive to the size of the tree. Therefore, it is preferable to also look at the following:

1. The `Gini index`:

$$ G = \sum_k \hat{p}_{mk}(1 - \hat{p}_{mk}) $$

2. Or the `Entropy` level:
    
$$ D = - \sum_k \max_k \hat{p}_{mk}\log(\hat{p}_{mk}) $$

## Classification Trees

- [Gini index](https://en.wikipedia.org/wiki/Gini_coefficient): measure of *node purity*: 

    - Small values indicate that all $\hat{p}_{mk}$ are either close to zero or one.

- [Entropy](https://en.wikipedia.org/wiki/Entropy):

    - Easy to see that: $0 \leq -\hat{p}_{mk}\log(\hat{p}_{mk})$
    - It will take values close to zero if $\hat{p}_{mk}$ is close to either zero or one.

## Classification Trees

- Feel free to use the measurement that you prefer.

- But, minimizing error rates could be preferable when classification accuracy is the target.

- But please cross-validate your analysis to pick the best parameters/tree size.

## Classification Trees

Classification in the book's heart disease data (before pruning):

![tree7](https://github.com/umbertomig/POLI175public/blob/main/img/tree7.png?raw=true)

## Classification Trees

Classification in the book's heart disease data (after pruning):

![tree8](https://github.com/umbertomig/POLI175public/blob/main/img/tree8.png?raw=true)

## Decision Trees x Linear Models

- Linear regression:

$$ f(X) \ = \ \beta_0 + \sum_{j=1}^p X_j\beta_j $$

- Decision Trees:

$$ f(X) \ = \ \sum_{m=1}^M c_m \cdot \mathbb{1}(X \in R_m) $$

- Which model is better? No easy answer.

## Decision Trees x Linear Models

Regression is better:

![tree9](https://github.com/umbertomig/POLI175public/blob/main/img/tree9.png?raw=true)

## Decision Trees x Linear Models

Decision tree is better:

![tree10](https://github.com/umbertomig/POLI175public/blob/main/img/tree10.png?raw=true)

## Decision Trees

- **Pros**:

    1. Easy to explain.

    2. Some argue that it mirrors human decision-making.

    3. Allow for graphical display (kind of pretty, if you ask me...).

    4. Easily handle qualitative predictors.

## Decision Trees

- **Cons**:

    1. Do not have the same level of accuracy as some predictive regression models.
    
    2. Can be *non-robust*: small changes in data make up to significant changes in final estimation.

## Decision Trees

- Decision trees for classification and regression tend to do poorly.

- This is because they are sensitive to the variation in the data.

- And this is something that you cannot change by rescaling the data:
    - Trees are **not** sensitive to the data scale.

## Decision Trees

- But here is the thing: We can combine many poor models and create a great one!

- We can combine many great models and create an even better one, but sometimes it is hard to improve on something already great.

- This technique is called **ensemble**!

## Civil War Dataset

In [ ]:
# Load data
dat = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/mshk-pa-2017/SambanisImp.csv')

## Target
target = "warstds"

## Predictors
predictors = ["ager", "agexp", "anoc", "army85", "autch98", "auto4","autonomy", "avgnabo", 
              "centpol3", "coldwar", "decade1", "decade2","decade3", "decade4", "dem", 
              "dem4", "demch98", "dlang", "drel", "durable", "ef", "ef2", "ehet", "elfo", 
              "elfo2", "etdo4590", "expgdp", "exrec", "fedpol3", "fuelexp", "gdpgrowth", 
              "geo1", "geo2", "geo34", "geo57", "geo69", "geo8", "illiteracy", "incumb", 
              "infant", "inst", "inst3", "life", "lmtnest", "ln_gdpen", "lpopns", "major", 
              "manuexp", "milper", "mirps0", "mirps1", "mirps2", "mirps3", "nat_war", 
              "ncontig", "nmgdp", "nmdp4_alt", "numlang", "nwstate", "oil", "p4mchg", 
              "parcomp", "parreg", "part", "partfree", "plural", "plurrel", "pol4", "pol4m", 
              "pol4sq", "polch98", "polcomp", "popdense", "presi", "pri", "proxregc", 
              "ptime", "reg", "regd4_alt", "relfrac", "seceduc", "second", "semipol3", "sip2", 
              "sxpnew", "sxpsq", "tnatwar", "trade", "warhist", "xconst"]

dat = dat[[target] + predictors]
y = dat[target]
X = dat[predictors]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y, random_state = 12345)

## Decision Trees

In [ ]:
## Decision Tree
dt = DecisionTreeClassifier(max_depth = 5)

# Fit dt to the training set
dt.fit(X_train, y_train)

# See Decision Tree (does not look great...)
fig = plt.figure(figsize = (25,20))
plot_tree(dt, fontsize = 15, feature_names = list(X.columns), 
          max_depth = 5, class_names = ['Peace', 'War'],
         label = 'root', filled = True)
plt.show()

## Decision Trees

In [ ]:
## Seeing it in text (not great either)
print(export_text(dt, feature_names = list(X.columns)))

## Decision Trees

In [ ]:
# Predict test set labels
y_pred = dt.predict(X_test)

# Compute test set accuracy
print(confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

# Confusion Matrix
ConfusionMatrixDisplay.from_estimator(dt, X_test, y_test,
        cmap = plt.cm.Blues, normalize = 'true')
plt.show()

## Decision Trees

In [ ]:
# Logistic Regression
logreg = LogisticRegression(max_iter = 100000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

# Compute test set accuracy  
accuracy_score(y_pred, y_test)

# Confusion Matrix
ConfusionMatrixDisplay.from_estimator(logreg, X_test, y_test,
        cmap = plt.cm.Blues, normalize = 'true')
plt.show()

## Ensemble

- Check an example of an ensemble using three classifiers:

In [ ]:
## Classifiers
logreg = LogisticRegression(max_iter = 10000)
lda = LinearDiscriminantAnalysis()
dt = DecisionTreeClassifier(max_depth = 10)
classifiers = [('Logistic Regression', logreg), ('LDA', lda), ('Classification Tree', dt)]

In [ ]:
# Each classifier
for clf_name, clf in classifiers:
    clf.fit(X_train, y_train)    
    y_pred = clf.predict(X_test)
    print('{} accuracy = {:.3f}'.format(clf_name, accuracy_score(y_test, y_pred)))

In [ ]:
# Their Ensemble
vc = VotingClassifier(estimators = classifiers)     
vc.fit(X_train, y_train)   
y_pred = vc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Voting Classifier: {:.3f}'.format(accuracy))

In [ ]:
# Confusion Matrix
ConfusionMatrixDisplay.from_estimator(vc, X_test, y_test,
        cmap = plt.cm.Blues, normalize = 'true')
plt.show()

## Detour: Bootstrap

- To understand some ensemble techniques, we must first learn what a bootstrap is.

- [**Bootstrap**](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)): Technique to fit models empirically, without deriving theoretically the parameters of interest.
    + We use it a lot to find standard errors and run things like [*exact tests*](https://en.wikipedia.org/wiki/Exact_test) and [randomization inference](https://dimewiki.worldbank.org/Randomization_Inference).

- Very empirical!

## Detour: Bootstrap

**Algorithm:** Start with the number of repetitions, N. For each $1, 2, \cdots, N$ step:

1. Draw a sample of the dataset [**with replacement**](https://en.wikipedia.org/wiki/Resampling_(statistics)) that has the same size of the dataset.

2. Fit the model (e.g., a regression) in the randomly drawn dataset.

3. Save the coefficient of interest.

In the end, take the mean of the coefficient as the `bootstrapped` coefficient and the standard deviation as the standard error.

## Bagging

- Bagging stands for Bootstrap Aggregation.

- The idea is to fit each tree on a bootstrapped dataset, then take the average of all trees.

- Each tree performs poorly. However, the average performance of all of them is better!

$$ \hat{f}_{bag}(x) \ = \ \dfrac{1}{B}\sum_{b = 1}^B \hat{f}^b(x) $$

## Bagging

- We let trees grow wildly: no pruning!
    + Averaging them out is what reduces the variance!
    
- For continuous variables, we take averages as the predicted value.

- How about classification problems?
    + Majority vote for all trees!

## Bagging

![bag1](https://github.com/umbertomig/POLI175public/blob/main/img/bag1.png?raw=true)

## Bagging

- This straightforward technique decreases the variance of a tree significantly.

- But how to interpret what the average of the trees means?
    - Well, we lose in terms of interpretation...

- One positive thing is that we can still find the **importance of each variable for the *bagging***

## Bagging

![bag2](https://github.com/umbertomig/POLI175public/blob/main/img/bag2.png?raw=true)

## Bagging

- Cross-validation here can be improved by using something called **out-of-bag** errors:
    + The bootstrap process usually leaves out 1/3 of the sample.
    + We can take advantage of this left-out (or *out-of-bag* sample), to estimate our models.

- And we fit RMSE for continuous or accuracy for discrete.

## Bagging

In [ ]:
## Bagging (discrete target)
dt = DecisionTreeClassifier()
bc = BaggingClassifier(estimator = dt, n_estimators = 50)
bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
print('Test set accuracy of bc: {:.2f}'.format(accuracy_score(y_test, y_pred))) 

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
## Bagging with OOB
bc = BaggingClassifier(estimator = dt, 
            n_estimators = 50,
            oob_score = True)
bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
acc_test = accuracy_score(y_test, y_pred)
acc_oob = bc.oob_score_

# Print acc_test and acc_oob
print('Test set accuracy: {:.3f}, OOB accuracy: {:.3f}'.format(acc_test, acc_oob))

## Women in Parliament Dataset

In [ ]:
## Countries Dataset
countr = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/countrdat.csv')
y2 = countr['wdi_wip']
X2 = countr[['wdi_expedu', 'pwt_pop', 'mad_gdppc']]
X2 = X2.join(pd.get_dummies(countr.ccodealp, drop_first = True, prefix = 'country'))
X2 = X2.join(pd.get_dummies(countr.year, drop_first = True, prefix = 'year'))
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.3, random_state = 12345)
X2.head()

## Women in Parliament Dataset

**Check-in:** Explore the WIP dataset.

In [ ]:
## Your code here

## Bagging

In [ ]:
## Bagging (continuous target)
dt = DecisionTreeRegressor()
bc = BaggingRegressor(estimator = dt, n_estimators = 50)
bc.fit(X2_train, y2_train)
y2_pred = bc.predict(X2_test)

# Lin reg (comparison)
lin = LinearRegression()
lin.fit(X2_train, y2_train)
y2_pred_lin = lin.predict(X2_test)
linreg_rmse = mean_squared_error(y2_test, y2_pred_lin) ** 0.5
bag_rmse = mean_squared_error(y2_test, y2_pred) ** 0.5

In [ ]:
# Comparing
print('Test set RMSE of Linear Regression is: {:.3f}'.format(linreg_rmse))
print('Test set RMSE of Bagging is: {:.3f}'.format(bag_rmse))

## Random Forests

- It is **not** a place where Computational Social Scientists go camping (this joke is getting old...)

- Random forests intend to improve the effectiveness of our bagging estimates.

- Each bagging tree in the ensemble can be highly correlated with each other.
    - This messes up the prediction because it reduces the contribution of each tree.

- To fix that, we tweak the bagging to *decorrelate* the trees.

## Random Forests

- A simple way to do that is only to consider a subset of the predictors at each tree.
    - Why would we even want to do that?
    
- Let a strong predictor with a bunch of other weak ones. Then:
    1. All bagging trees will rely on the stronger predictor more than the others.
    2. Subsetting the number of variables, considering subsets where the strong predictor is not there, improves the usage of the weak predictors.
        - This *decorrelates* the trees!

- Rule of thumb: Use $m = \sqrt{p}$ predictors at a time.

## Random Forests

- Choose a small(er) $m$ if the predictors are all highly correlated.

![img](https://github.com/umbertomig/POLI175public/blob/main/img/rf1.png?raw=true)

## Random Forests

In [ ]:
# Random Forests (discrete target)
rf = RandomForestClassifier(n_estimators = 100, random_state = 12345)  
rf.fit(X_train, y_train) 
y_pred = rf.predict(X_test)

# Print accuracy
print('Test set Accuracy of Random Forest: {:.2f}'.format(accuracy_score(y_test, y_pred)))

## Random Forests

In [ ]:
# Confusion Matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
ConfusionMatrixDisplay.from_estimator(rf, X_test, y_test,
        cmap = plt.cm.Blues, normalize = 'true')
plt.show()

## Random Forests

In [ ]:
# Create a pd.Series of features importances
importances = pd.Series(data = rf.feature_importances_,
                        index = X_train.columns)

# Sort importances
importances_sorted = importances.sort_values()

# Draw a horizontal barplot of importances_sorted
fig = plt.figure(figsize = (25,20))
importances_sorted.plot(kind='barh', color='lightgreen')
plt.title('Features Importances')
plt.show()

## Random Forests

In [ ]:
# Random Forests (continuous variable)
rf = RandomForestRegressor(n_estimators = 25, random_state = 12345)  
rf.fit(X2_train, y2_train)
y2_pred = rf.predict(X2_test)

# Evaluate the test set RMSE
rf_rmse = mean_squared_error(y2_test, y2_pred) ** 0.5

# Print rmse_test
print('Test set RMSE of Linear Regression is: {:.5f}'.format(linreg_rmse))
print('Test set RMSE of Bagging is: {:.3f}'.format(bag_rmse))
print('Test set RMSE of Random Forest is: {:.5f}'.format(rf_rmse))

## Random Forests

In [ ]:
# Create a pd.Series of features importances
importances = pd.Series(data = rf.feature_importances_,
                        index = X2_train.columns)

# Sort importances
importances_sorted = importances.sort_values()

# Draw a horizontal barplot of importances_sorted
fig = plt.figure(figsize = (25,20))
importances_sorted.tail(10).plot(kind='barh', color='lightgreen')
plt.title('Features Importances')
plt.show()

## Boosting

- Ensemble method that combines weak learners to form a stronger one.
    + Example: Regression tree that is only allowed to have one leaf!

- It builds on accumulation: Every predictor tries to improve the predecessor's job.
    - Work with the errors of the previous models, and update the fit slowly.

## Boosting

**Algorithm:** Start with a null model ($\hat{f}(x) = 0$), the residual equals to $r_i = y_i$, and a number $B$ of steps.

For each $b \in \{1, 2, \cdots, B\}$:

1. Fit a tree $\hat{f}^b(x)$ with $d$ splits (or d+1 terminal nodes).

2. Set:

$$ \hat{f}_{new}(x) = \hat{f}_{old}(x) + \lambda \hat{f}^b(x) $$

3. Set 

$$ r_{i_{new}} = r_{i_{old}} - \lambda \hat{f}^b(x) $$

At the end, you should define $\hat{f}(x)$ as:

$$ \hat{f}(x) = \sum_{b=1}^B \lambda \hat{f}^b(x) $$

## Boosting

- You can overfit using boosting. But only if $B$ is too large.

- $\lambda$: Controls the rate that your boosting algorithm is learning.
    + Small $\lambda$s require large $B$s

- $d$: Controls the complexity of each step. $d=1$ tends to work well!

## Boosting

![imgb](https://upload.wikimedia.org/wikipedia/commons/b/b5/Ensemble_Boosting.svg)

## Boosting

In [ ]:
# AdaBoosting (discrete target)
dt = DecisionTreeClassifier(max_depth = 2)
ada = AdaBoostClassifier(estimator = dt, n_estimators = 250, random_state = 12345)
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)
print('Test set Accuracy of AdaBoosting: {:3f}'.format(accuracy_score(y_test, y_pred)*100))

## Boosting

In [ ]:
# Confusion Matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
ConfusionMatrixDisplay.from_estimator(ada, X_test, y_test,
        cmap = plt.cm.Blues, normalize = 'true')
plt.show()

## Boosting

In [ ]:
# AdaBoosting (continuous variable)
regtree = DecisionTreeRegressor(max_depth = 2)
ada = AdaBoostRegressor(regtree, n_estimators = 25, random_state = 12345)  
ada.fit(X2_train, y2_train)
y2_pred = ada.predict(X2_test)

# Evaluate the test set RMSE
ada_rmse = mean_squared_error(y2_test, y2_pred) ** 0.5

# Print rmse_test
print('Test set RMSE of Linear Regression is: {:.5f}'.format(linreg_rmse))
print('Test set RMSE of Bagging is: {:.3f}'.format(bag_rmse))
print('Test set RMSE of Random Forest is: {:.5f}'.format(rf_rmse))
print('Test set RMSE of AdaBoosting is: {:.5f}'.format(ada_rmse))

## Boosting

In [ ]:
# GradientBoost (discrete target)
gbr = GradientBoostingClassifier(max_depth = 5, n_estimators = 250, random_state = 12345)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
print('Test set Accuracy of Gradient Boosting: {:3f}'.format(accuracy_score(y_test, y_pred)*100))

## Boosting

In [ ]:
# Gradient Boosting (continuous variable)
gbr = GradientBoostingRegressor(n_estimators = 25, random_state = 12345)  
gbr.fit(X2_train, y2_train)
y2_pred = gbr.predict(X2_test)

# Evaluate the test set RMSE
gbr_rmse = mean_squared_error(y2_test, y2_pred) ** 0.5

# Print rmse_test
print('Test set RMSE of Linear Regression is: {:.5f}'.format(linreg_rmse))
print('Test set RMSE of Bagging is: {:.3f}'.format(bag_rmse))
print('Test set RMSE of Random Forest is: {:.5f}'.format(rf_rmse))
print('Test set RMSE of AdaBoosting is: {:.5f}'.format(ada_rmse))
print('Test set RMSE of Gradient Boosting is: {:.5f}'.format(gbr_rmse))

## Ensemble Methods

**Check-in:** Use ensemble methods to try to predict swing voters.

In [ ]:
anes = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/anes2016.csv')

# Questions?

# See you in the next class!